## How To Find Patients Meeting Specific Criteria

In [1]:
from ClinicalDocumentUtils import Database

In [2]:
d = Database()

Connect To Database


encryption key: ········


Connected: {'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'clinical', 'host': 'clinical_db', 'port': '5432', 'options': '-c search_path=clinical_document,public', 'sslmode': 'require', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'}


## Get list of projects and their UUID

In [3]:
d.fetch_projects()

,name,description,uuid
0,test,test,00e50190-f573-4750-8946-70b70f2ca70c


### Set Project UUID

In [4]:
d.set_project_uuid('00e50190-f573-4750-8946-70b70f2ca70c')

project_uuid: 00e50190-f573-4750-8946-70b70f2ca70c


## Data Structure
To understand how to extract information, you should understand the basic 4D data structure of ClinicalDocument. 
![alt text](./images/datastructure.png)


## Patient Filter Parameters

The filter is an array of json objects. The structure follows:
```
filter = [ {'icd10':<string>, 'tag':<string>, 'attribute':<string>, 'value':<array>},...]
```
You can see from the above data structure, to obtain mrns you just have to specify the filter.

### Example 1: 
Find all patient mrns that have a dob tag.


In [5]:
filters = [
{'icd10':None, 'tag': 'dob'},
]
d.get_mrns_where_filters(filters)

['s222222', 's111111']

### Example 2:
Find all patient mrns that have BOTH 'diagnosis' AND 'dob' tags

In [8]:
patient_filter=[{'icd10':None, 'tag': 'dob'},
                {'icd10':None, 'tag': 'diagnosis'}]

print(f"{patient_filter=}")
print()
print(f"mrns:{d.get_mrns_where_filters(patient_filter)}")

patient_filter=[{'icd10': None, 'tag': 'dob'}, {'icd10': None, 'tag': 'diagnosis'}]

mrns:['s222222', 's111111']


### Example 3:
Find all patients that have tags 'dob' AND 'T' with the attribute 'T'

In [39]:
patient_filter=[{'icd10':None, 'tag': 'dob'},
                {'icd10':'c61', 'tag': 'pT', 'attribute':'pT' }]

print(f"{patient_filter=}")
print()
print(f"mrns:{d.get_mrns_where_filters(patient_filter)}")

patient_filter=[{'icd10': None, 'tag': 'dob'}, {'icd10': 'c61', 'tag': 'pT', 'attribute': 'pT'}]

mrns:['333333' '111111' '666666' '777777']


### Example 4:
Find all patients with the tag 'diagnosis' with the attribute named 'value' with the value "c61 - Malignant neoplasm of prostate"

In [40]:
patient_filter=[{'icd10':None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}]
print(f"{patient_filter=}")
print()
print(f"mrns:{d.get_mrns_where_filters(patient_filter)}")
print()

patient_filter=[{'icd10': None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}]

mrns:['333333' '888888' '111111' '555555' '666666' '777777']



### Example 5:

'%' is a wildcard
An underscore (_) in pattern stands for (matches) any single character.\
Reference: https://www.postgresql.org/docs/current/functions-matching.html

Find patient with a the diagnosis of 'c61 - Malignant neoplasm of prostate' AND and pT-stage including 'T2a', and 'T3a'

In [41]:
# Structure [{tag:<string>, attribute:<string>, value:<array>},...]
# Searches for patients with the diagnosis: "C61;Malignant_neoplasm_of_prostate" AND whose T stage is 'Tx' OR 'T3'
patient_filter=[{'icd10':None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}, 
                  {'icd10':'c61', 'tag': 'pT', 'attribute': 'pT', 'value': ['T2c', 'T3a']},]
print(f"{patient_filter=}")
print()
print(f"mrns:{d.get_mrns_where_filters(patient_filter)}")


patient_filter=[{'icd10': None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}, {'icd10': 'c61', 'tag': 'pT', 'attribute': 'pT', 'value': ['T2c', 'T3a']}]

mrns:['333333' '111111' '666666' '777777']


### Example 6:

'%' is a wildcard
An underscore (_) in pattern stands for (matches) any single character
Reference: https://www.postgresql.org/docs/current/functions-matching.html

Find patient with a the diagnosis of 'c61 - Malignant neoplasm of prostate' AND and any pT-stage starting with T3

In [42]:
# Structure [{tag:<string>, attribute:<string>, value:<array>},...]
# Searches for patients with the diagnosis: "C61;Malignant_neoplasm_of_prostate" AND whose T stage is 'Tx' OR 'T3'
patient_filter=[{'icd10':None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}, 
                  {'icd10':'c61', 'tag': 'pT', 'attribute': 'pT', 'value': ['T3%']},]
print(f"{patient_filter=}")
print()
print(f"mrns:{d.get_mrns_where_filters(patient_filter)}")

patient_filter=[{'icd10': None, 'tag': 'diagnosis', 'attribute': 'diagnosis', 'value': ['c61 - Malignant neoplasm of prostate']}, {'icd10': 'c61', 'tag': 'pT', 'attribute': 'pT', 'value': ['T3%']}]

mrns:['333333' '666666']


## Close Database Class

In [43]:
del d